In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from config import api_key
import json
from pprint import pprint

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("Output Data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,1.79,75,20,1.54,AR,1717955224
1,1,albany,42.6001,-73.9662,19.46,84,86,0.89,US,1717954645
2,2,punta arenas,-53.1500,-70.9167,2.06,80,75,1.54,CL,1717955149
3,3,port moresby,-9.4431,147.1797,24.29,91,13,4.26,PG,1717955226
4,4,cannonvale,-20.2833,148.7000,18.66,77,0,2.65,AU,1717955226


### Step 1: Create a map that displays a point for every city in the city_data_df DataFrame as shown in the following image. The size of the point should be the humidity in each city.

In [7]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    fill_alpha = 0.5,
    title = "Plot showing cities from city_data_df",
    xlabel = "Latitude",
    ylabel = "Longitude"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition.

In [8]:
# Narrow down cities that fit criteria and drop any results with null values

# Set my ideal weather conditions to filter the data
temp_min = 20
temp_max = 35
win_speed_max = 5
cloudiness_max = 30
humidity_max = 50

# Filter the city_data_df DataFrame for the ideal weather conditions
ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= temp_min) &
                                   (city_data_df["Max Temp"] <= temp_max) &
                                   (city_data_df["Wind Speed"] <= win_speed_max) &
                                   (city_data_df["Cloudiness"] <= cloudiness_max) &
                                   (city_data_df["Humidity"] <= humidity_max)]

# Drop any rows with null values
ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,fort bragg,35.1390,-79.0060,34.90,40,20,4.63,US,1717955227
53,53,hamilton,39.1834,-84.5333,25.60,50,20,4.63,US,1717955229
80,80,takai,11.5833,9.1167,34.88,35,16,3.39,NG,1717955231
82,82,luderitz,-26.6481,15.1594,20.58,32,17,3.95,NaN,1717955231
107,107,huayin,34.5675,110.0921,23.85,46,20,3.93,CN,1717955232
154,154,sur,22.5667,59.5289,33.10,47,13,3.54,OM,1717955236
196,196,brookings,44.3114,-96.7984,23.38,37,0,3.09,US,1717955070
205,205,diamantina,-18.2494,-43.6003,22.58,45,26,4.06,BR,1717955239
208,208,pokhara,28.2333,83.9833,25.54,37,26,3.46,NP,1717955196
242,242,big bear city,34.2611,-116.8450,24.73,14,0,4.47,US,1717955242


### Step 3: Create a new DataFrame called hotel_df to store the city, country, coordinates, and humidity.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
21,fort bragg,US,35.1390,-79.0060,40,
53,hamilton,US,39.1834,-84.5333,50,
80,takai,NG,11.5833,9.1167,35,
82,luderitz,NaN,-26.6481,15.1594,32,
107,huayin,CN,34.5675,110.0921,46,
154,sur,OM,22.5667,59.5289,47,
196,brookings,US,44.3114,-96.7984,37,
205,diamantina,BR,-18.2494,-43.6003,45,
208,pokhara,NP,28.2333,83.9833,37,
242,big bear city,US,34.2611,-116.8450,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":api_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address_json = name_address.json()

        # Note: I had used pprint to display the response to confirm details
        # pprint(name_address_json)

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fort bragg - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
takai - nearest hotel: No hotel found
luderitz - nearest hotel: No hotel found
huayin - nearest hotel: No hotel found
sur - nearest hotel: No hotel found
brookings - nearest hotel: No hotel found
diamantina - nearest hotel: No hotel found
pokhara - nearest hotel: No hotel found
big bear city - nearest hotel: No hotel found
mulayjah - nearest hotel: No hotel found
jinzhou - nearest hotel: No hotel found
canoinhas - nearest hotel: No hotel found
diego de almagro - nearest hotel: No hotel found
nuevo ideal - nearest hotel: No hotel found
kaithal - nearest hotel: No hotel found
al minya - nearest hotel: No hotel found
ondjiva - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
ayvacik - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
21,fort bragg,US,35.1390,-79.0060,40,No hotel found
53,hamilton,US,39.1834,-84.5333,50,No hotel found
80,takai,NG,11.5833,9.1167,35,No hotel found
82,luderitz,NaN,-26.6481,15.1594,32,No hotel found
107,huayin,CN,34.5675,110.0921,46,No hotel found
154,sur,OM,22.5667,59.5289,47,No hotel found
196,brookings,US,44.3114,-96.7984,37,No hotel found
205,diamantina,BR,-18.2494,-43.6003,45,No hotel found
208,pokhara,NP,28.2333,83.9833,37,No hotel found
242,big bear city,US,34.2611,-116.8450,14,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city on the map

In [12]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2.0,
    color = "City",
    fill_alpha = 0.5,
    title = "Plot showing ideal cities for vacation",
    xlabel = "Latitude",
    ylabel = "Longitude",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)